# HW04 Multiple Regression and Creating the Real Estate Dataset

As of now we have been creating the model and trying to test if model is good or not using some metrics. In this homework we will not just create the model but we will perform one most important steps that needs to be done before creating the model that is preparing the dataset.

There is one popoular data science workflow mostly famous in R programming. The work flow is given by Hadley wickham in his book "R for Data Science".The work flow give the following steps :
1. Data Collection
2. Data Import
3. Data Tidy
4. Data Transform
5. Data Visualize
6. Data Model
7. Data Communication

In this homework we will be focusing on the data transform steps along with data visualization and data modeling as we all know these are iterative process which we will continue to do unitl we get the desired result.

Before we start as we always do, we will import the libraries that we will be using in this homework.


*Importing Libraries*



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
import seaborn as sns
plt.style.use("ggplot")

Next steps is to import the dataset that we will be using in the homework.


In [2]:
df_realestate = pd.read_csv("data/Real Estate Data - Week 4.csv",index_col=0,header=0)


,Dwelling Type,Lot Frontage,Lot Area,Alley,Land Contour,Neighborhood,Location Condition,Overall Quality,Overall Cond,Year Built,...,Garage Finish,Garage Cars,Wood Deck Area,Open Porch Area,Enclosed Porch Area,3 Season Porch Area,Screen Porch Area,Pool Area,Sale Condition,Sale Price
Id,,,,,,,,,,,,,,,,,,,,,
1,2-STORY 1946 & NEWER,65.0,8450,NaN,Level,College Creek,Normal,7,5,2003,...,Rough Finished,2,0,61,0,0,0,0,Normal Sale,208500
2,1-STORY 1946 & NEWER,80.0,9600,NaN,Level,Veenker,Adjacent Feeder St,6,8,1976,...,Rough Finished,2,298,0,0,0,0,0,Normal Sale,181500
3,2-STORY 1946 & NEWER,68.0,11250,NaN,Level,College Creek,Normal,7,5,2001,...,Rough Finished,2,0,42,0,0,0,0,Normal Sale,223500
4,2-STORY 1945 & OLDER,60.0,9550,NaN,Level,Crawford,Normal,7,5,1915,...,Unfinished,3,0,35,272,0,0,0,"Abnormal Sale - trade, foreclosure, short sale",140000
5,2-STORY 1946 & NEWER,84.0,14260,NaN,Level,Northridge,Normal,8,5,2000,...,Rough Finished,3,192,84,0,0,0,0,Normal Sale,250000


## 5. Create the Real State Dataset.

### Section 5a: Drop features that are not needed

Task in hand:
1. Drop the features that are not needed.
2. Drop `Alley` and `Garage Yr Build` as they have too many null values which are not possible to impute.
3. Drop `Year Built` because year built is normally not a factor in determining the price of the house.Which is partially true as Year can be significant factor for cost due to use of technology and materials in construction and some old house might cost more due to its antique value. So cannot just drop it based on this assumption.But for this homework we are going to drop it.
4. Drop `Basement Area` as `Basement Area` = `Basement Unfinished Area` + `Basement Finished Area`, therefore it would be redundant to include all 3. 

In [5]:
# Drop unnecessary features from the dataset
cols_to_drop = [
    'Alley',
    'Garage Yr Built',  # Corrected column name
    'Year Built',
    'Basement Area'
]
df_realestate = df_realestate.drop(columns=cols_to_drop)
df_realestate.head()

,Dwelling Type,Lot Frontage,Lot Area,Land Contour,Neighborhood,Location Condition,Overall Quality,Overall Cond,Masonry/Veneer Area,Basement Finished Area,...,Garage Finish,Garage Cars,Wood Deck Area,Open Porch Area,Enclosed Porch Area,3 Season Porch Area,Screen Porch Area,Pool Area,Sale Condition,Sale Price
Id,,,,,,,,,,,,,,,,,,,,,
1,2-STORY 1946 & NEWER,65.0,8450,Level,College Creek,Normal,7,5,196.0,706,...,Rough Finished,2,0,61,0,0,0,0,Normal Sale,208500
2,1-STORY 1946 & NEWER,80.0,9600,Level,Veenker,Adjacent Feeder St,6,8,0.0,978,...,Rough Finished,2,298,0,0,0,0,0,Normal Sale,181500
3,2-STORY 1946 & NEWER,68.0,11250,Level,College Creek,Normal,7,5,162.0,486,...,Rough Finished,2,0,42,0,0,0,0,Normal Sale,223500
4,2-STORY 1945 & OLDER,60.0,9550,Level,Crawford,Normal,7,5,0.0,216,...,Unfinished,3,0,35,272,0,0,0,"Abnormal Sale - trade, foreclosure, short sale",140000
5,2-STORY 1946 & NEWER,84.0,14260,Level,Northridge,Normal,8,5,350.0,655,...,Rough Finished,3,192,84,0,0,0,0,Normal Sale,250000


### Section 5b: Fill Missing Values in select features
Task in hand:
1. When we want to include a feature that has null values, we will need to fill in the NaN's with a value that makes sense to do so.
2. For `Lot Frontage`, we will replace NaN's with zero as we are assuming this.
3. Same with the `Masonry/Veneer Area`.
4. For the `Porch Area`, we will create a new feature by adding up `Open Porch Area`, `Enclosed Porch Area`, and `3-Season Porch Area` into one new feature and remove the old features.
5. Combine all Full Baths into one `Full Bathrooms` feature by adding up `Basement Full Baths` and `Full Baths Above Grade`.
6. Combine all Half Baths into one `Half Bathrooms` feature by adding up `Basement Half baths` and `Half Baths Above Grade`. Drop the features that were used to create `Porch Area`, `Full Bathrooms`, and `Half Bathrooms`.

In [ ]:
# 1-3. Fill missing values for selected features
# Replace NaN's in 'Lot Frontage' and 'Masonry/Veneer Area' with zero
for col in ['Lot Frontage', 'Masonry/Veneer Area']:
    df_realestate[col] = df_realestate[col].fillna(0)

# 4. Create 'Porch Area' by summing porch-related features
# 5. Combine all Full Baths into one 'Full Bathrooms' feature
# 6. Combine all Half Baths into one 'Half Bathrooms' feature

df_realestate['Porch Area'] = (
    df_realestate['Open Porch Area'] +
    df_realestate['Enclosed Porch Area'] +
    df_realestate['3 Season Porch Area']
)
df_realestate['Full Bathrooms'] = (
    df_realestate['Basement Full Baths'] +
    df_realestate['Full Baths Above Grade']
)
df_realestate['Half Bathrooms'] = (
    df_realestate['Basement Half baths'] +
    df_realestate['Half Baths Above Grade']
)

# Drop the original columns used to create the new features
cols_to_drop = [
    'Open Porch Area', 'Enclosed Porch Area', '3 Season Porch Area',
    'Basement Full Baths', 'Full Baths Above Grade',
    'Basement Half baths', 'Half Baths Above Grade'
]
df_realestate = df_realestate.drop(columns=cols_to_drop)
df_realestate.head()